### Game Description:

Winter is here. You and your friends were tossing around a frisbee at the park when you made a wild throw that left the frisbee out in the middle of the lake. The water is mostly frozen, but there are a few holes where the ice has melted. If you step into one of those holes, you'll fall into the freezing water. At this time, there's an international frisbee shortage, so it's absolutely imperative that you navigate across the lake and retrieve the disc. However, the ice is slippery, so you won't always move in the direction you intend.   

The surface is described using a grid like the following:  
  
    SFFF       (S: starting point, safe)
    FHFH       (F: frozen surface, safe)
    FFFH       (H: hole, fall to your doom)
    HFFG       (G: goal, where the frisbee is located)
    
The episode ends when you reach the goal or fall in a hole. You receive a reward of 1 if you reach the goal, and zero otherwise.

### Elements of Markov Decision Process (MDP):
>**Environment** :  The Grid of size (4,4) is the environment (Frozen Lake).

>**Agent**: The Software Thing which interactcs with the environment. Agent's starting position in the environment is state 'S'.

>**State**: There are 16 states available. state 'S' is starting point in the environment. state 'F' is frozen surface. state 'H' is the hole and state 'G' is the final state or terminate state.

>**Action**: 4 Possible actions which agent can take are - left, right, up and down.

>**Reward**: Agent would earn reward of 1 when it reaches the terminate state('G') else for eevry other state agent would get reward 0.

### Q-Learning Algorithm.
Q-Learning is a model-free Reinforcement learning algorithm uses MDP framework.It involves an agent, set of states **S**, and a set **A** of actions per state. Agent Can perform an action **$a\in A$** from a specific state **$s\in S$** and recahes to a new state **$s^{'} \in S$** and also earns a reward **R**.

Parameters used in the algorithm are -

> **Discounting Factor ($\gamma$):** It is used to solve the problem in an environment where states are infinite. Agent's goal in Q-Learning RL is to maximize the return (total rewards throughout all steps in an episode). But when states are infinite total rewards also becomes infinite. To address this a discounting factor is added to each future reward . discounting factor gives less importance to immediate reward over future reward and helps agents to get finite total rewards. $\gamma$ can be between 0 to 1. We will set the parameter value to 0.99.($\gamma = 0.99$).

>**Learning rate ($\alpha$):** In Q-Learning algo a Q-Table of state action pairs is maintained to hold the Q-value for each state action pair. Q-value in each state gets updated everytime when agent  take specific action from specific state. alpha is a measure of how quickly to adopt the new Q-value. when alpha is 1 old Q-value gets removed completly. if it is between 0 to 1 old Q-values gets updated slowly with new Q-value.

>**Exploration Rate ($\epsilon$):** This tells agent to explore or exploit. when agent starts initially it should explore the environment means it should learn to take action and understand the environment. when agent plays through the environment more it should exploit (use) the information already available to it.There should be a trade off between exploration and exploitation. epsilon is the tradeoff. it can be between 0 to 1. $\epsilon == 1$ meand 100% probability that agent would explore the environment meaning it would take random action in the environment.

>**Number of Episoded**: When agent starts from initial state and reaches to final state it is called 1 episode. in a game like frozen lake an agent can take multiple episodes meaning agent can reach to its goal state many times through different state action pairs. We will allow our agent to learn by completing 10000 such episodes.

>**Number of States**: An agent can reach its goal by transitioning from one state to another. we will restrict our agent to reach its goal by taking maximum 100 steps per episodes.If agent does not reach its goal state within 100 steps then it would earn nothing. 


Lets write the highlevel pseudo code for the implementation of 'Frozen-Lake' game using Q-Learning RL algorithm.

1. create environment object.  
2. initialize all parameters to be used.  
3. initialize q-table of shape (states_count , action_count).    
4. Train agent as follows  


    for each episode :
        set state to first state.
        set reward to 0
    
        for each step:
            find action
            perform action

            update q-table with new q-value for state action pair
            update state to new state
            update earned reward

            if final_state:
                break

        update epsilon
    store reward per episode
        

In [1]:
# Importing required libraries
import gym
import numpy as np
import random
import time
from IPython.display import clear_output

In [2]:
#Creating 'Frozen Lake' environment built by OpenAI-gym
env = gym.make('FrozenLake-v0')

In [3]:
print('Count of Possible Actions ' , env.action_space.n)
print('Count of Possible States ', env.observation_space.n)

Count of Possible Actions  4
Count of Possible States  16


In [4]:
#Createing Q-Table and Initializing

acion_space_size_ = env.action_space.n
state_space_size_ = env.observation_space.n

q_table = np.zeros((state_space_size_, acion_space_size_)) 
print('Initial Q-Table: \n', q_table)

Initial Q-Table: 
 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [5]:
# Initializing Q-Learning Parameters
gamma_ = 0.99
alpha_ = 0.1

epsilon_ = 1
max_epsilon_ = 1
min_epsilon_ = 0.01
epsilon_decay_ = 0.001

episode_len_ = 10000
max_steps_ = 100

rewards_per_episode_ = []

In [6]:
#Q-Learning Algorithm - Training

for ep in range(episode_len_):
    state = env.reset()
    done = False
    curr_reward = 0
    
    for stp in range(max_steps_):
        
        #Exploration-Exploitation Trade Off
        exploration_threshold = random.uniform(0,1)
        if exploration_threshold > epsilon_:
            action = np.argmax(q_table[state,:])
        else:
            action = env.action_space.sample()
        
        #Performing Action
        new_state, reward, done, info = env.step(action)
        
        #Update the Q-Table
        q_table[state, action] = (1 - alpha_)*q_table[state, action] + \
                                alpha_ * ( reward + gamma_ * np.max(q_table[new_state, :]) )
        
        #Updating Current Reward
        curr_reward += reward
        #print('expl_th ',exploration_threshold)
        #print('state ',state)
        #print('action ',action)
        #print('reward ', curr_reward)
        # Updating State to new state
        state = new_state
        
        # Break if agent reached terminate state
        if done == True:
            break
    
    # Decaying Exploration Rate
    epsilon_ = min_epsilon_ + (max_epsilon_ - min_epsilon_)* np.exp(- epsilon_decay_ * ep)
    
    # Storing the Reward achieved in an episode
    rewards_per_episode_.append(curr_reward)
    

Training our agent is completed in above code. Now lets see how agent performed. We will look at the average rewards per thousands episode.

In [7]:
# Check Average reward per thousand episode
rewards_per_thousand_episode = np.split(np.array(rewards_per_episode_),(episode_len_/1000))
count = 1000
for rewards in rewards_per_thousand_episode:
    print(count ,':', (sum(rewards)/1000))
    count += 1000

1000 : 0.034
2000 : 0.197
3000 : 0.41
4000 : 0.545
5000 : 0.62
6000 : 0.631
7000 : 0.675
8000 : 0.671
9000 : 0.667
10000 : 0.679


We can see that average reward is increased over time. when agent completed first 1000 episodes ,average reward was 3% and when agent completed 10000 episodes , rewards jumps to 67%.

Our agent played 10,000 episodes. At each time step within an episode, the agent received a reward of 1 if it reached the frisbee, otherwise, it received a reward of 0. If the agent did indeed reach the frisbee, then the episode finished at that time-step. So, that means for each episode, the total reward received by the agent for the entire episode is either 1 or 0. So, for the first thousand episodes, we can interpret this score as meaning that 3% of the time, the agent received a reward of 1 and won the episode. And by the last thousand episodes from a total of 10,000, the agent was winning 67% of the time.  

Lets print the final updated Q-Table for state action pair.

In [8]:
print('Updated Q-Table after 10000 Episodes \n', q_table)

Updated Q-Table after 10000 Episodes 
 [[0.54945633 0.52607016 0.50171397 0.52952858]
 [0.3685751  0.31978314 0.3551354  0.49970314]
 [0.41680771 0.39045468 0.41030492 0.45702724]
 [0.24228146 0.28574767 0.27535768 0.44670587]
 [0.57520308 0.43207537 0.3774571  0.35430742]
 [0.         0.         0.         0.        ]
 [0.2977282  0.14329652 0.15935138 0.08978597]
 [0.         0.         0.         0.        ]
 [0.38025314 0.37792853 0.49447403 0.65553935]
 [0.51896736 0.6995394  0.45060328 0.29917972]
 [0.68305296 0.41437302 0.25256196 0.38059724]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.41199595 0.60804558 0.76155089 0.47475399]
 [0.75630301 0.88079808 0.70657757 0.72878231]
 [0.         0.         0.         0.        ]]


Below lines of code is just to visualise the game .

In [12]:
# Testing Our Agent - Visualising
for episode in range(20):
    state = env.reset()
    done = False
    print('**********EPISODE****** ', episode+1)
    
    for step in range(max_steps_):
        clear_output(wait=True)
        env.render()
        time.sleep(0.5)
        
        action = np.argmax(q_table[state,:])
        new_state , reward, done, info = env.step(action)
        
        if done:
            clear_output(wait=True)
            env.render()
            if reward == 1:
                print("****You reached the goal!****")
                time.sleep(3)
            else:
                print("****You fell through a hole!****")
                time.sleep(3)
                clear_output(wait=True)
            break
        state = new_state
env.close()

  (Down)
SFFF
FHFH
FFFH
HFFG
****You reached the goal!****
